In [ ]:
# from google.colab import drive
# import os
# import sys

# drive.mount('/content/drive')
# sys.path.append('/content/drive/MyDrive/Grammer_Correction/')

# os.chdir('/content/drive/MyDrive/Grammer_Correction/')

In [1]:
import random
from enum import Enum
import re
import numpy as np
from pyarabic.araby import separate, tokenize, is_arabicrange, strip_tashkeel, strip_tatweel
from gensim.models import Word2Vec, FastText,KeyedVectors
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
# from torchtext.data import Field, BucketIterator
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from scripts import *
from Seq2seq import *


file_path_alignments = "./gec-datasets/alignments/"
file_path_datasets = "./gec-datasets/"

# from scripts_ged.ged_scripts.ged_preprocessor import *

cpu


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [3]:
import gc
source_list1, target_list1 = read_alignments(file_path_alignments + 'qalb14/qalb14_train.txt')
source_list1, target_list1 = create_windows(source_list1,target_list1,window_size = 5)
print(len(source_list1))

source_list2, target_list2 = read_alignments(file_path_alignments + 'qalb15/qalb15_train.txt')
source_list2, target_list2 = create_windows(source_list2,target_list2,window_size = 5)
print(len(source_list2))

source_list3, target_list3 = read_alignments(file_path_alignments + 'zaebuc/zaebuc_train.txt')
source_list3, target_list3 = create_windows(source_list3,target_list3,window_size = 5)
print(len(source_list3))

source_list = []
target_list = []

source_list += (source_list1 + source_list2 + source_list3)

target_list += (target_list1 + target_list2 + target_list3)

del source_list1 , source_list2, source_list3, target_list1, target_list2, target_list3
gc.collect()
print(len(source_list))




220079
9210
4890
234179


In [4]:
print(source_list[-7])
print(target_list[-7])
print(len(target_list))

['<SOS>', 'المضلم', 'فهوا', 'تعلم', 'أشياء', 'خاطئة', '<EOS>']
['<SOS>', 'المظلم', 'فهو', 'تعلم', 'أشياء', 'خاطئة', '<EOS>']
234179


## Word2vec Finetuning

In [5]:
arabic_list = read_arabic_for_word2vec(file_path_datasets + "splitted_qalb14/train.txt")
arabic_list2 = read_arabic_for_word2vec(file_path_datasets + "splitted_qalb15/qalb15_train.txt")
arabic_list3 = read_arabic_for_word2vec(file_path_datasets + "splitted_zaebuc/zaebuc_train.txt")
arabic_list4 = read_arabic_for_word2vec(file_path_datasets  + "splitted_qalb14/dev.txt")
arabic_list5 = read_arabic_for_word2vec(file_path_datasets  + "splitted_qalb14/test.txt")
arabic_list6 = read_arabic_for_word2vec(file_path_datasets  + "splitted_zaebuc/dev_train.txt")
arabic_list.extend(arabic_list2)
arabic_list.extend(arabic_list3)
arabic_list.extend(arabic_list4)
arabic_list.extend(arabic_list5)
arabic_list.extend(arabic_list6)
# add <SOS> and <EOS> tokens at beginning and end of each sentence in arabic_list where each sentence is a list of tokens
for i in range(len(arabic_list)):
  arabic_list[i] = ['<SOS>'] + arabic_list[i] + ['<EOS>']

arabic_list.extend(target_list)


word2vec_model = Word2Vec(sentences=arabic_list, vector_size=256, window=5, min_count=0, workers=4, sg=1)

## Embedding matrix creation


In [6]:
all_tokens = source_list + target_list
word_to_index = create_word_to_index(all_tokens)
index_to_word = {idx: word for word, idx in word_to_index.items()}
vocab_size = len(word_to_index)
print(vocab_size)
word_to_index['المظلم']

165142


43304

In [7]:
combined_vocab = set(word_to_index.keys())
print(len(combined_vocab))

165142


In [8]:
source_to_index = create_word_to_index(source_list)
print(len(source_to_index))
source_vocab_size = len(source_to_index)
print(source_to_index['المضلم'])

141069
125501


In [9]:
target_to_index = create_word_to_index(target_list)
print(len(target_to_index))
target_vocab_size = len(target_to_index)
print(target_to_index['المظلم'])
index_to_word_target = {idx: word for word, idx in target_to_index.items()}

111469
37131


In [10]:
word2vec_vocab = set(word2vec_model.wv.key_to_index.keys())

In [11]:
import pickle

with open('./pickle_files/source2index.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(source_to_index, file)
    
with open('./pickle_files/target2index.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(target_to_index, file)

    
with open('./pickle_files/index2word_target.pkl', 'wb') as file:
    # Serialize the dictionary and write it to the file
    pickle.dump(index_to_word_target, file)
    
    

In [12]:
source_vocab = set(source_to_index.keys())
print(len(source_vocab))


141069


In [13]:
target_vocab = set(target_to_index.keys())
print(len(target_vocab))

111469


In [14]:
def create_embedding_matrix(vocab, vocab_size, embedding_dim, word_to_index, word2vec_model,word2vec_vocab):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    unk = 0
    mean = np.mean(word2vec_model.wv.vectors, axis=0)
    for word in vocab:
        # if word in glove_vocab:
        #     embedding_matrix[word_to_index[word]] = glove_model[word]
        if word == '<SPACE>':
            embedding_matrix[word_to_index[word]] = np.zeros(embedding_dim)
        elif word == '<PAD>':
            embedding_matrix[word_to_index[word]] = np.ones(embedding_dim)
        elif word in word2vec_vocab:
            embedding_matrix[word_to_index[word]] = word2vec_model.wv[word]
        else:
            # <UNK> token
                unk+=1
                embedding_matrix[word_to_index[word]] = mean

    print(unk)
    return embedding_matrix
        

In [15]:
source_embedding_matrix = create_embedding_matrix(source_vocab,source_vocab_size, 256, source_to_index, word2vec_model, word2vec_vocab)
target_embedding_matrix = create_embedding_matrix(target_vocab,target_vocab_size, 256, target_to_index, word2vec_model, word2vec_vocab)


27810
1


In [16]:
source_embedding_matrix.shape

(141069, 256)

In [17]:
target_embedding_matrix.shape

(111469, 256)

## Data preperation

In [18]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, source, target,source_pad_index = source_to_index['<PAD>'],target_pad_index = target_to_index['<PAD>']):
        self.source = source
        self.target = target
        self.max_length = max(max(len(s) for s in source), max(len(t) for t in target))

        self.source = torch.tensor([s + [source_pad_index] * (self.max_length - len(s)) for s in source])
        self.target = torch.tensor([t + [target_pad_index] * (self.max_length - len(t)) for t in target])

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

In [19]:
print(source_list[-1],target_list[-1])

['<SOS>', 'إستعمالهم', 'لها', '.', '<EOS>'] ['<SOS>', 'استعمالهم', 'لها', '.', '<EOS>']


In [20]:
source_list = convert_to_indicies(source_list,source_to_index)
target_list = convert_to_indicies(target_list,target_to_index)


In [21]:
print(source_list[-1],target_list[-1])

[0, 141066, 736, 20, 1] [0, 110242, 726, 20, 1]


In [22]:
batch_size = 5
mini_sentences = source_list[0: 8]
mini_labels = target_list[0: 8]
mini_dataset = Dataset(mini_sentences, mini_labels)
dummy_dataloader = torch.utils.data.DataLoader(mini_dataset, batch_size=5)
dg = iter(dummy_dataloader)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

torch.Size([5, 7]) torch.Size([5, 7]) torch.Size([3, 7]) torch.Size([3, 7])
tensor([0, 2, 3, 4, 5, 6, 1]) 
 tensor([0, 2, 3, 4, 5, 6, 1])


In [23]:
dataset = Dataset(source_list, target_list)

## Training Hyperparamaters

In [24]:
num_epochs = 10
batch_size = 64
learning_rate = 0.001

## Model Hyperparamaters


In [25]:
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_input_size = source_vocab_size
decoder_input_size = target_vocab_size
embedding_size = 256
hidden_size = 1024
num_layers = 2
dropout_p = 0.5
print(device)


cpu


In [26]:
encoder = EncoderLSTM(
    encoder_input_size,
    embedding_size,
    source_embedding_matrix,
    hidden_size,
    num_layers,
    dropout_p
).to(device)

In [27]:
decoder = DecoderLSTM(
    decoder_input_size,
    embedding_size,
    target_embedding_matrix,
    hidden_size,
    num_layers,
    dropout_p,
).to(device)

In [28]:
model = Seq2Seq(encoder, decoder).to(device)
print(model)


Seq2Seq(
  (encoder): EncoderLSTM(
    (embedding): Embedding(141069, 256)
    (lstm): LSTM(256, 1024, num_layers=2, dropout=0.5)
    (dropout): SpatialDropout(p=0.5, inplace=False)
  )
  (decoder): DecoderLSTM(
    (embedding): Embedding(111469, 256)
    (lstm): LSTM(256, 1024, num_layers=2, dropout=0.5)
    (out): Linear(in_features=1024, out_features=111469, bias=True)
    (dropout): SpatialDropout(p=0.5, inplace=False)
  )
)


## Training loop

In [29]:
def eval_fun(training_example,model):
    lis = [word_to_index.get(word,source_to_index['<UNK>']) for word in training_example]
    tensor = torch.tensor(lis, dtype=torch.long)
    tensor = tensor.unsqueeze(1)
    tensor = tensor.to(device)
    output = model.inference(tensor,tensor.shape[0])
    output = output.reshape(-1, output.shape[2])
    output = output.argmax(dim=1)
    final = []
    for indx in output:
        final.append(index_to_word_target[indx.item()])
    
    return final
        
   

In [30]:
def train(model, train_dataset, training_example, batch_size=10, epochs=5, learning_rate=0.01,model_name = "model"):
  """
  This function implements the training logic
  Inputs:
  - model: the model ot be trained
  - train_dataset: the training set of type Dataset
  - batch_size: integer represents the number of examples per step
  - epochs: integer represents the total number of epochs (full training pass)
  - learning_rate: the learning rate to be used by the optimizer
  """
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    for source, targets in tqdm(train_dataloader):
      source = source.transpose(0,1)
      targets = targets.transpose(0,1)
      source = source.to(device)
      targets = targets.to(device)
      # print(source.shape)
      # print(targets.shape)
      output = model(source,targets)
      # print("Output of Seq2seq ", output.shape)
      output = output[1:].reshape(-1, output.shape[2]) # to neglect start token
      # print("Output of Seq2seq after reshaping ", output.shape)
      # print("target shape ", targets.shape)
      targets = targets[1:].reshape(-1) # to neglect start token
      # print("target shape after reshaping ", targets.shape)

      batch_loss = criterion(output, targets)
      total_loss_train += batch_loss.item()
      # print("shape of argmax ", output.argmax(dim=1).shape)
      acc = (output.argmax(dim=1) == targets).sum().item()
      total_acc_train += acc

      optimizer.zero_grad()
      batch_loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
      optimizer.step()

    epoch_loss = total_loss_train / len(train_dataset)
    epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0][0]))
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')
    print("sentence evaluation",eval_fun(training_example,model))
    
  torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, model_name + '_checkpoint.pth')  

In [31]:
test =  ['<SOS>', 'الشي', 'عن', 'برزة', 'وانا', 'ببرزة', '<EOS>']

In [ ]:
train(model,dataset,test,batch_size,num_epochs,learning_rate)